In [2]:
from typing import Callable, Dict, List, Any
import pandas as pd
import numpy as np
import sqlite3
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures._base import Future
import os
from string import ascii_uppercase
import pickle
import re
from sql_extract import extract_lines, lapply, clean_sql_file, ref_dict, id_lines
from pprint import pprint

In [ ]:
def clean_sql_file(input_file_path: str, output_file_path: str):
    input_file = open(input_file_path, 'r') 
    output_file = open(output_file_path, "w")
    while True:
        try:
            line = input_file.readline()
            line = line.replace("\'","#!").replace("'", "\"")
            output_file.write(line)
            if line == '':
                break
        except:
            pass
    input_file.close() 
    output_file.close()

In [ ]:
ef clean_sql_file(input_file_path: str, output_file_path: str):
    input_file = open(input_file_path, 'r') 
    output_file = open(output_file_path, "w")
    while True:
        try:
            line = input_file.readline()
            
            output_file.write(line)
            if line == '':
                break
        except:
            pass
    input_file.close() 
    output_file.close()

In [37]:
"\"esf"

'"esf'

In [ ]:
data = pd.read_csv("Data/pairs_with_ratings.tsv", sep="\t").drop(['uid'], axis=1)
data

In [ ]:
print(data.isna().sum())
data.describe()

# SQL data
## Loading the data

In [2]:
with open('tables.pkl', 'rb') as f:
    tables = pickle.load(f) # deserialize using load()

## Cleaning the data

In [3]:
import sqlite3

# Step 1: Specify the path to the .sql file and the .db file
sql_file_path = "unfun_2023-02-02.sql"  # The .sql file containing SQL commands
db_file_path = 'new_test.db'  # The SQLite .db file to be created
nb_errors = 0
# Step 2: Connect to SQLite (it will create the .db file if it doesn't exist)
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

# Step 3: Read the .sql file containing SQL commands
try:
    with open(sql_file_path, 'r') as sql_file:
        sql_script = sql_file.read()

    # Step 4: Execute the SQL script to create tables, insert data, etc.
    cursor.executescript(sql_script)
    print("SQL script executed successfully, and the SQLite database has been created.")
except FileNotFoundError as e:
    print(f"SQL file not found: {e}")
except sqlite3.Error as e:
    print(f"An error occurred while executing the SQL script: {e}")
    nb_errors+=1

finally:
    # Step 5: Commit changes and close the connection
    conn.commit()
    conn.close()
print("\n##############################################################\n")
print(f"Nb errors: {nb_errors}")


An error occurred while executing the SQL script: near "s": syntax error

##############################################################

Nb errors: 1


In [4]:
# Extracting all sql commands for relevent tables
commands = {
"create_original" : [],
"lines_original" : [],
"create_unfunned" : [],
"lines_unfunned" : [],
"create_ratings" : [],
"lines_ratings" : []
}
cursor_list = list(commands.keys())
cursor = len(cursor_list)
idx = 0
with open("unfun_2023-02-02_test.sql", 'r') as file:
    for _ in range(10):
        next(file)
        idx+=1
        print(idx,end="\r")
    for line in file:
        if line[:40].find("CREATE TABLE `headlines_original`") != -1:
            cursor = 0
        elif line[:40].find("INSERT INTO `headlines_original`") != -1:
            cursor = 1
        elif line[:40].find("CREATE TABLE `headlines_unfunned`") != -1:
            cursor = 2
        elif line[:40].find("INSERT INTO `headlines_unfunned`") != -1:
            cursor = 3
        elif line[:40].find("CREATE TABLE `ratings`") != -1:
            cursor = 4
        elif line[:40].find("INSERT INTO `ratings`") != -1:
            cursor = 5
        elif line == "\n":
            cursor = len(cursor_list)
        if cursor != len(cursor_list):
            commands[cursor_list[cursor]].append(line)
        idx+=1
        print(f"% {idx} | O: {len(commands["lines_original"])} | U: {len(commands["lines_unfunned"])} | R: {len(commands["lines_ratings"])}",end="\r")
print("\n")

% 144065 | O: 0 | U: 0 | R: 0



In [3]:
with open('failed_comands.pkl', 'rb') as f:
    errors = pickle.load(f) # deserialize using load()

In [41]:
temp = """CREATE TABLE "headlines_original" (
  "id" int(11) NOT NULL,
  "title" varchar(512) NOT NULL,
  "truth_type" varchar(45) NOT NULL,
  "domain" varchar(45) NOT NULL,
  "url" varchar(512) NOT NULL,
  "date" datetime DEFAULT NULL
)  ;"""
cursor_2023.execute(temp)

In [50]:
for i in errors:
    print(i)

lines_original
lines_unfunned
lines_ratings


In [58]:
errors["lines_original"][0]

'\nINSERT INTO "headlines_original" ("id", "title", "truth_type", "domain", "url", "date") VALUES\n(10201, \'league cup semi-final: bradford go bonkers at final whistle\', \'real\', \'bbc.co.uk\', \'http://bbc.co.uk/sport/0/football/21151984\', \'2013-01-22 22:10:14\'),\n(10202, \'neighbour tried to douse burning teenager\', \'real\', \'bbc.co.uk\', \'http://news.bbc.co.uk/go/rss/-/1/hi/scotland/tayside_and_central/8368692.stm\', \'2009-11-19 15:37:54\'),\n(10203, \'road collapses into a cavern below\', \'real\', \'bbc.co.uk\', \'http://news.bbc.co.uk/go/rss/-/2/hi/europe/jersey/8368573.stm\', \'2009-11-19 15:38:10\'),\n(10204, \'when does weighing up all options become dithering?\', \'real\', \'bbc.co.uk\', \'http://news.bbc.co.uk/go/rss/-/2/hi/uk_news/politics/8364283.stm\', \'2009-11-19 15:39:25\'),\n(10205, \'tractor run to flood-hit school\', \'real\', \'bbc.co.uk\', \'http://news.bbc.co.uk/go/rss/-/2/hi/uk_news/northern_ireland/foyle_and_west/8368631.stm\', \'2009-11-19 15:39:44\

In [30]:
errors["lines_original"][0].split("\n")[2][:-1]

"(10201, 'league cup semi-final: bradford go bonkers at final whistle', 'real', 'bbc.co.uk', 'http://bbc.co.uk/sport/0/football/21151984', '2013-01-22 22:10:14')"

In [13]:
errors["lines_original"][0][42298-50:42298+150]

" '2011-09-08 12:30:53'),\n(10423, 'muslim militant group releases video claiming deadly western china attacks', 'real', 'foxnews.com', 'http://foxnews.com/world/2011/09/08/muslim-militant-group-release"

In [18]:
import sqlite3
SQL_con_2023 = sqlite3.connect("test.db")
cursor_2023 = SQL_con_2023.cursor()

In [8]:
cursor_2023.execute(errors["lines_original"][0].replace("\n",""))
result = cursor_2023.fetchall()

OperationalError: near "group": syntax error

In [ ]:
tables["lines_original"]

In [14]:
np.array(list(tables.keys()))[["line" in string for string in list(tables.keys())]]

array(['lines_original', 'lines_unfunned', 'lines_ratings'], dtype='<U15')

In [ ]:
("".join(tables["lines_original"])).split(";")

In [16]:
("".join(tables["lines_original"])).split(");")[5].find("seriously")

779

In [22]:
("".join(tables["lines_original"])).split(");")[5][760:800]

" chief: al qaeda \\'seriously degraded\\' "

In [17]:
idx = 0
tab_ref = "lines_original"
pos = ("".join(tables[tab_ref])).split(");")[errors[tab_ref][idx][0]].find(errors[tab_ref][idx][1])
("".join(tables[tab_ref])).split(");")[errors[tab_ref][idx][0]][pos-10:800]

' `headlines_original'

In [28]:
("".join(tables[tab_ref])).split(");")[0][pos-20:pos+20]

'NSERT INTO `headlines_original` (`id`, `'

In [ ]:
("".join(tables[tab_ref])).split(";")[0].split("\n")

In [7]:
SQL_con_2023 = sqlite3.connect("Unfunned_data.db")
cursor_2023 = SQL_con_2023.cursor()

In [8]:
get_list_tables = """SELECT name FROM sqlite_master WHERE type='table';"""
request = """ SELECT DISTINCT O.id, O.title, U.title, R.rating, U.date
              FROM headlines_original O, headlines_unfunned U, ratings R
              WHERE O.id = U.original_headline_id 
              and R.id = O.id
              and U.title is not null;

"""
cursor_2023.execute(request)
result = cursor_2023.fetchall()

In [13]:
test = pd.read_sql(request, con= SQL_con_2023)

In [14]:
test.to_csv()

,id,title,title,rating,date
0,23881,Chase Utley Takes Seventh-Inning Stretch Too S...,CHASE UTLEY TAKES SEVENTH-WINNING STRETCH TOO ...,1.000,2014-08-18 01:37:21
1,18481,Depressed Groundhog Sees Shadow Of Rodent He O...,DEPRESSED GROUNDHOG SEES SHADOW OF THE RODENT ...,0.000,2014-08-18 01:38:05
2,22583,Wendy's To Phase Out Unpopular Hamburger Sandwich,WENDY'S TO PHASE OUT UNPOPULAR HAMBURGER SANDW...,0.000,2014-08-18 01:39:00
3,22583,Wendy's To Phase Out Unpopular Hamburger Sandwich,WENDY'S TO PHASE OUT UNPOPULAR lamb SANDWICH,0.000,2014-08-18 08:41:40
4,21557,Brendan Fraser To Star In New Pre-Movie Trivia...,BRENDAN FRASER TO STAR IN NEW PRE-MOVIE TRIVIA...,0.903,2014-08-18 08:45:45
...,...,...,...,...,...
11627,20251,U.S. Signs Declaration Of Dependence On China,U.S. SIGNS TRADE DECLARATION WITH CHINA,NaN,2020-12-08 16:09:01
11628,20266,Overheating Planet That Will Ultimately End Hu...,OVERHEATING PLANET THAT WILL ULTIMATELY END HU...,1.000,2020-12-08 16:13:04
11629,21234,FDA Approves Sale Of Prescription Placebo,FDA APPROVES SALE OF PRESCRIPTION PENICILIN,0.956,2020-12-08 16:23:14
11630,26818,Report: Most Americans Canâ€™t Even Name Their...,REPORT: MOST AMERICANS CANâ€™T EVEN NAME THEIR...,0.828,2020-12-08 16:23:39


In [ ]:
pd.DataFrame()

In [88]:
("".join(tables["create_ratings"])).replace("ENGINE=InnoDB DEFAULT CHARSET=utf8", "").replace("\n","")

'CREATE TABLE `ratings` (  `id` int(11) NOT NULL,  `uid` varchar(95) NOT NULL,  `batch_id` varchar(32) NOT NULL,  `headline_id` int(11) NOT NULL,  `headline_type` varchar(32) NOT NULL,  `date` datetime NOT NULL,  `position` int(11) NOT NULL,  `rating` double DEFAULT NULL) ;'

In [17]:
import linecache 
particular_line = linecache.getline("unfun_2023-02-02.sql", 50809) 


In [20]:
particular_line.replace('\\"', '"')

'(9812, "internacional - cia "contrat?" a blackwater contra al-qaeda", "real", "bbc.co.uk", "http://bbc.co.uk/mundo/internacional/2009/08/090820_1200_eeuu_cia_blackwater_ms.shtml", "2009-08-20 15:01:00"),\n'

In [17]:
replacement_dict = {"â€˜" : "\"",
                    "â€™" : "\""}

In [6]:
test = pd.read_csv("unfun_2023.csv").rename(columns={"title":"original_tile", "title.1":"unfuned_title"})

In [25]:
test.iloc[0,3]

'CHASE UTLEY TAKES SEVENTH-WINNING STRETCH TOO SERIOUSLY, REPORT TEAMMATES'

In [26]:
for i in range(len(test["id"])):
    for key in replacement_dict:
        try:
            test.iloc[i,2] = test.iloc[i,2].replace(key,replacement_dict[key])
        except:
            pass
        try:
            test.iloc[i,3] = test.iloc[i,3].replace(key,replacement_dict[key])
        except:
            pass


0
CHASE UTLEY TAKES SEVENTH-WINNING STRETCH TOO SERIOUSLY, REPORT TEAMMATES
0
CHASE UTLEY TAKES SEVENTH-WINNING STRETCH TOO SERIOUSLY, REPORT TEAMMATES
1
DEPRESSED GROUNDHOG SEES SHADOW OF THE RODENT HE ONCE WAS
1
DEPRESSED GROUNDHOG SEES SHADOW OF THE RODENT HE ONCE WAS
2
WENDY'S TO PHASE OUT UNPOPULAR HAMBURGER SANDWITCH
2
WENDY'S TO PHASE OUT UNPOPULAR HAMBURGER SANDWITCH
3
WENDY'S TO PHASE OUT UNPOPULAR lamb SANDWICH
3
WENDY'S TO PHASE OUT UNPOPULAR lamb SANDWICH
4
BRENDAN FRASER TO STAR IN NEW PRE-MOVIE TRIVIA Show
4
BRENDAN FRASER TO STAR IN NEW PRE-MOVIE TRIVIA Show
5
President personally attends first obamacare operation
5
President personally attends first obamacare operation
6
Obama to create new jobs by redefining restaurant policies
6
Obama to create new jobs by redefining restaurant policies
7
BRENDAN FRASER TO STAR IN NEW MOVIE TRIVIAL QUESTIONS
7
BRENDAN FRASER TO STAR IN NEW MOVIE TRIVIAL QUESTIONS
8
NEW JACKIE ROBINSON MOVIE HAS VIOLENT SCENE
8
NEW JACKIE ROBINSON MOVI

AttributeError: 'float' object has no attribute 'replace'

In [19]:
line = "â€˜New York Timesâ€™ Article Refers To Him As â€˜Mr. Hitlerâ€™"
for key in replacement_dict:
    line = line.replace(key,replacement_dict[key])
line

'"New York Times" Article Refers To Him As "Mr. Hitler"'

In [12]:
test.iloc[0,2]

'Chase Utley Takes Seventh-Inning Stretch Too Seriously, Report Teammates'

In [9]:
starts = np.where(lines_types == ref_dict["Start_original"])[0]
ends = np.where(lines_types == ref_dict["Comand_end"])[0]

In [13]:
lines[70636]

np.str_('(24989),\n')

In [10]:
min(ends[ends > max(starts)])

np.int64(70363)

In [45]:
temp = ends[ends > min(starts)]
#temp = temp[temp < np.where(temp == min(ends[ends > max(starts)]))[0]]


In [44]:
min(ends[ends > max(starts)])

np.int64(70363)

In [ ]:
temp[:np.where(temp == min(ends[ends > max(starts)]))[0][0]+1]

In [38]:
temp

array([], dtype=int64)

In [ ]:
(ends > min(starts)) and (ends < max(starts))

In [129]:
lines[70185]

np.str_("(29266, 'American Airlines Drops Bereavement Fares', 'real', 'theonion.com', 'http://www.theonion.com/articles/american-airlines-drops-bereavement-fares,35438/', '2014-03-05 00:00:00');\n")

In [134]:
def extract_entires(lines_type_array: List[int], table_ref: str, ref_dict: Dict[str, int]) -> List[int]:
    nb_inserts = np.where(lines_type_array == ref_dict[table_ref])[0] # Get all the lines for the insert comand
    out = np.array([], dtype=int)
    for idx in range(len(nb_inserts)-1):
        start = nb_inserts[idx]
        end = nb_inserts[idx+1]
        print(f"start: {start}, end: {end}")
        entries = np.where(lines_type_array == ref_dict["Entry"])[0]
        out = np.append(out, entries[(entries > start) == (entries < end)])
        out = np.append(out, entries[end])
    return out
        

In [ ]:
test = extract_entires(lines_types, "Start_original", ref_dict)

In [136]:
lines[test]

array(["(9332, 'sunday feature: a brief history of cunning', 'real', 'bbc.co.uk', 'http://bbc.co.uk/go/rss/-/iplayer/episode/b00cx18z', '2008-08-13 00:06:38'),\n",
       "(9333, 'olympics - all programmes - page 1', 'real', 'bbc.co.uk', 'http://bbc.co.uk/go/rss/-/iplayer/categories/olympics', '2008-08-13 00:06:39'),\n",
       "(9334, 'house of saddam: episode 3', 'real', 'bbc.co.uk', 'http://bbc.co.uk/go/rss/-/iplayer/episode/b00d1y00', '2008-08-15 17:24:00'),\n",
       ...,
       "(29264, 'New Theme Park To Feature Model Of Noahâ€™s Ark', 'real', 'theonion.com', 'http://www.theonion.com/articles/new-theme-park-to-feature-model-of-noahs-ark,35429/', '2014-03-03 00:00:00'),\n",
       "(29265, 'Pope Francis Drops F-Bomb During Vatican Blessing', 'real', 'theonion.com', 'http://www.theonion.com/articles/pope-francis-drops-fbomb-during-vatican-blessing,35437/', '2014-03-04 00:00:00'),\n",
       "(268, '504504090138', '8af62193b5bb39419aa2a5bf10591ba5', 22494, NULL, '2014-08-13 18:47:

In [106]:
test

array([50328, 50329, 50330, ..., 70179, 70180, 70916])

In [108]:
lines

array(['-- phpMyAdmin SQL Dump\n', '-- version 4.9.7\n',
       '-- https://www.phpmyadmin.net/\n', ..., '\n', '--\n', ''],
      dtype='<U652')

In [ ]:
nb_inserts = np.where(lines_types == ref_dict["Start_original"]) # Get all the lines for the insert comand
print(nb_inserts)

In [70]:
out = []
for idx in range(len(nb_inserts)-1):
    start = nb_inserts[idx]
    end = nb_inserts[idx+1]
    print(f"start: {start}, and: {end}")
    entries = np.where(lines_types == ref_dict["Entry"])[0]
    print(entries)
    out.extend(entries[(entries > start) == (entries < end)])

In [72]:
len(nb_inserts)

1

In [71]:
range(len(nb_inserts)-1)

range(0, 0)

In [64]:
entries = np.where(lines_types == ref_dict["Entry"])[0]
len(entries[(entries > 50327) == (entries < 50627)])

298

In [44]:
(entries > 50327 and entries < 50627

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [48]:
test = np.array([1,2,3,4,5,6,7,8,9,10])

In [60]:
(test > 1) == (test < 8)

array([False,  True,  True,  True,  True,  True,  True, False, False,
       False])

In [56]:
(test > 1 & test < 8).any()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [28]:
np.where(lines == "ERROR")

(array([59738, 68013, 69084, 70150, 70756, 71489, 76271, 83666, 93156]),)

In [90]:
len("INSERT INTO `ratings`")

21

In [56]:
test= lapply(filter_line, lines)

In [36]:
print(out)

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [ ]:
def filter_line(line: str):
    if line[0] not in ascii_uppercase + '(':
        return ""
    return line

test= apply_function_concurrently(lines[:5], filter_line)


In [16]:

lines = []
flag =False
with open('unfun_2023-02-02.txt') as file:

    for line in file:
        if not flag:
            if line.find("INSERT INTO `batches` (`id`, `uid`, `date`, `http_user_agent`, `ip_address`) VALUES"):
                flag = True
        if flag:
            try:
                lines.append(line)
            except:
                lines.append("ERROR")


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1804: character maps to <undefined>